## Learning Lyapunov function for Circle Path Following

In [ ]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 

## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 2×2 identity matrix and R is 1×1 identity matrix

In [ ]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1,bias=False)
        self.control.weight = torch.nn.Parameter(lqr)

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = self.control(x)
        return out,u

## Dynamical system

In [ ]:
def f_value(x,u):
    v = 6
    l = 1
    y = [] 
    
    for r in range(0,len(x)): 
        f = [v*torch.sin(x[r][1]),
             v*torch.tan(u[r][0])/l -(torch.cos(x[r][1])/(1-x[r][0]))]
        y.append(f) 
    y = torch.tensor(y)    
    return y

## Options

In [ ]:
'''
For learning 
'''
N = 500            # sample size
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)  

lqr = torch.tensor([[-0.8471 , -1.6414]])  # lqr solution
x = torch.Tensor(N, D_in).uniform_(-1, 1)           
x_0 = torch.zeros([1, 2])
x = torch.cat((x, x_0), 0)

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
vars_ = [x1,x2]
v = 6
l = 1
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
epsilon = 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.2
ball_ub = 0.8

## Learning and Falsification

In [ ]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    lqr = torch.tensor([[-23.58639732,  -5.31421063]])    # lqr solution
    model = Net(D_in,H1, D_out,lqr)
    L = []
    i = 0 
    t = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        V_candidate, u = model(x)
        X0,u0 = model(x_0)
        f = f_value(x,u)
        Circle_Tuning = Tune(x)
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

        # With tuning
        Lyapunov_risk = (F.relu(-V_candidate)+ 2*F.relu(L_V+0.8)).mean()\
                    +1.5*((Circle_Tuning-V_candidate).pow(2)).mean()+ 1.2*(X0).pow(2)


        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        w1 = model.layer1.weight.data.numpy()
        w2 = model.layer2.weight.data.numpy()
        b1 = model.layer1.bias.data.numpy()
        b2 = model.layer2.bias.data.numpy()
        q = model.control.weight.data.numpy()

        # Falsification
        if i % 10 == 0:
            u_NN = (q.item(0)*x1 + q.item(1)*x2) 
            f = [v*sin(x2),
                 v*tan(u_NN)/l -(cos(x2)/(1-x1))]

            # Candidate V
            z1 = np.dot(vars_,w1.T)+b1

            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            V_learn = tanh(z2.item(0))

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            result= CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config,epsilon)
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = AddCounterexamples(x,result,10)
            else:  
                valid = True
                print("Satisfy conditions!!")
                print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        i += 1

    stop = timeit.default_timer()


    np.savetxt("w1_p.txt", model.layer1.weight.data, fmt="%s")
    np.savetxt("w2_p.txt", model.layer2.weight.data, fmt="%s")
    np.savetxt("b1_p.txt", model.layer1.bias.data, fmt="%s")
    np.savetxt("b2_p.txt", model.layer2.bias.data, fmt="%s")
    np.savetxt("q_p.txt", model.control.weight.data, fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

### Checking result with smaller epsilon

In [4]:
epsilon = -0.00001
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, epsilon)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
    print(result)
else:  
    print("Satisfy conditions with epsilon= ",epsilon)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

('Satisfy conditions with epsilon= ', -1e-05)
(<Expression "tanh((0.099664619999999995 + 0.6251369 * tanh((-1.2775723999999999 - 2.5249655 * x1 - 0.18407556 * x2)) - 1.0490067000000001 * tanh((-0.46410936000000003 - 0.47741896 * x1 - 0.69643679999999997 * x2)) - 1.0708085000000001 * tanh((-0.3698631 - 0.52394079999999998 * x1 - 0.58617014000000001 * x2)) + 0.46437746000000002 * tanh((0.91936094000000002 - 0.023182843000000002 * x1 - 0.50320260000000006 * x2)) + 0.70194889999999999 * tanh((0.97582674000000003 - 0.062661019999999998 * x1 - 0.56203824000000002 * x2)) - 1.1286860000000001 * tanh((1.3282121 + 1.3561938 * x1 + 2.5183802000000002 * x2))))">, ' is a Lyapunov function.')
